##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# <div dir="rtl">تمهيد سريع للخبراء حول Tensorflow 2</div>

<table class="tfo-notebook-buttons" align="right" dir="rtl">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/advanced"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />إفتح المحتوى على موقع TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />تفاعل مع المحتوى على Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />اعرض المصدر على Github</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />نزّل الدّفتر</a>
  </td>
</table>

<div dir="rtl">
هذا الملفّ هو دفتر 
<a href="https://colab.research.google.com/notebooks/welcome.ipynb">Google Colaboratory</a>.
بواسطته ، يمكنك تشغيل برامج Python مباشرة في المتصفّح - 
و هي طريقة رائعة لتعلّم و اِستخدام Tensorflow.

 لمتابعة هذا البرنامج التعليمي ، قم بتشغيل الدفتر التّفاعلي في 
 Google Colab
 بالنقر فوق الزر، ذي نفس التسمية ، الموجود في أعلى هذه الصفحة.
</div>

<div dir="rtl">

1. في Colab ، اتصل بمحرّك تشغيل Python بالطريقة التّالية: إذهب إلى قائمة الإختيارات في أعلى يسار الدفتر ، ثمّ إضغط على <i>CONNECT</i>.
2. شغّل كافة خلايا  الدفتر التّفاعلي بإختيار <i>Runtime</i> ثمّ الضغط على  <i>Run all</i>. 

</div>

<div dir="rtl">
قم بتنزيل وتثبيت TensorFlow 2. ثمّ قم بإستيراد حزمة TensorFlow في برنامجك:

ملاحظة: 
قم بتحديث 
pip
 لتتمكّن من تثبيت حزمة
Tensorflow 2.
أنظر في 
<a href="https://www.tensorflow.org/install">دليل التثبيت
</a>
للحصول على المزيد من التفاصيل. 
</div>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

<div dir="rtl">
اِسترد حزمة 
Tensorflow
في برنامجك
</div>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

<div dir="rtl">
قم بتحميل و إعداد 
<a href="http://yann.lecun.com/exdb/mnist/">
مجموعة بيانات MNIST
</a>
</div>

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

<div dir="rtl">
اِستخدم الدّالة
<code>tf.data</code>
لخلط مجموعة البيانات عشوائيًا
ثمّ
 توزيعها على دفعات ، كل منها متكوّن من 32 مثالاً
 :
</div>

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

<div dir="rtl">
قم ببناء نموذج 
<code>tf.keras</code>
باستخدام 
الفئات الفرعية للنماذج 
(<a href="https://www.tensorflow.org/guide/keras#model_subclassing">model subclassing API</a>)
الموجودة في واجهة برمجة 
Keras:
</div>


In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

<div dir="rtl">
لتدريب النموذج قم باختيار خوارزميّة تحسين 
(optimizer)
و دالّة خسارة  
(loss function)
:
</div>

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

<div dir="rtl">
حدّد المقاييس 
(metrics)
لقياس مقدار الخسارة 
(loss)
و مدى دقّة 
(accuracy)
النّموذج.
تُجمع قيم هذه المقاييس عبر فترات التّدريب
(epochs)
ثمّ تطبع النتيجة الجمليّة.   
</div>

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

<div dir="rtl">
اِستخدم الدالّة
<code>tf.GradientTape</code>
لتدريب النموذج:
</div>

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

<div dir="rtl">
اِختبر النموذج:
</div>

In [0]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

<div dir="rtl">
الآن و بعد التّدريب، بلغت دقة النموذج في تصنيف الصور الموجودة في مجموعة البيانات المستخدمة في هذا المقال إلى
%98 تقريبًا
.
لتعلّم المزيد ، اِقرأ 
<a href="https://www.tensorflow.org/tutorials/">
الدروس التعليمية
</a>
 الأخرى ل
TensorFlow.
</div>